In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import os
import pickle
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv("equipment_data.csv")

# Select features and targets
features = [
    "Total_Admissions_Today",
    "Total_Discharges_Today",
    "ICU_Admissions_Today",
    "Surgeries_Today",
    "Bed_Occupancy_Rate",
    "Is_Weekend"
]
targets = ["Patient_Monitors_Required", "Defibrillators_Required", "Infusion_Pumps_Required"]

# Normalize numerical features
scaler = MinMaxScaler()
df_scaled = df.copy()
df_scaled[features + targets] = scaler.fit_transform(df[features + targets])
df_scaled[features + targets] = df_scaled[features + targets].astype(np.float32)

# Train-Test Split (80% Train, 20% Test)
train_size = int(len(df_scaled) * 0.8)
train, test = df_scaled.iloc[:train_size], df_scaled.iloc[train_size:]

# Function to create sequences for LSTM
def create_sequences(data, seq_length=10):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length][features].values)  # Features
        y.append(data.iloc[i+seq_length][targets].values)  # Target
    return np.array(X), np.array(y)

# Prepare sequences
seq_length = 10  # Model looks at last 10 days
X_train, y_train = create_sequences(train, seq_length)
X_test, y_test = create_sequences(test, seq_length)

# Define LSTM Model
def build_lstm_model():
    model = Sequential([
        LSTM(128, return_sequences=True, input_shape=(seq_length, len(features))),
        Dropout(0.2),
        LSTM(64, return_sequences=True),
        Dropout(0.2),
        LSTM(32),
        Dropout(0.2),
        Dense(len(targets))  # Predicting multiple targets
    ])
    model.compile(loss='mse', optimizer='adam')
    return model

# Path for saving model
model_path = "lstm_equipment_prediction.pkl"

# Load or Train Model
if os.path.exists(model_path):
    with open(model_path, "rb") as f:
        model_json, weights = pickle.load(f)
    model = model_from_json(model_json)
    model.set_weights(weights)
    model.compile(loss='mse', optimizer='adam')
    print("Loaded existing model.")
else:
    model = build_lstm_model()
    model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test), verbose=1)

    # Save model
    model_json = model.to_json()
    weights = model.get_weights()
    with open(model_path, "wb") as f:
        pickle.dump((model_json, weights), f)

    print("Trained and saved new model.")

# Predict next day's equipment requirement
latest_data = df_scaled.iloc[-seq_length:][features].values.reshape(1, seq_length, len(features))
predicted_scaled = model.predict(latest_data)

# Reverse scale the predictions
predicted_full = np.zeros((1, len(features) + len(targets)))
predicted_full[:, -len(targets):] = predicted_scaled.flatten()
predicted_equipment = scaler.inverse_transform(predicted_full)[:, -len(targets):][0]  # Extract only target columns

# Print predictions
print(f"Predicted Equipment for Tomorrow:")
print(f"Patient Monitors: {predicted_equipment[0]:.2f}")
print(f"Defibrillators: {predicted_equipment[1]:.2f}")
print(f"Infusion Pumps: {predicted_equipment[2]:.2f}")

# Function to update the model with actual values
def update_model_with_actual(actual_values):
    """
    Updates the model with the new actual data at the end of the day.
    """
    global df_scaled, X_train, y_train

    # Add new actual data
    new_data = df.iloc[-1:].copy()
    for i, target in enumerate(targets):
        new_data[target] = actual_values[i]  # Use actual entered values

    # Normalize new data
    new_data_scaled = new_data.copy()
    new_data_scaled[features + targets] = scaler.transform(new_data[features + targets])

    # Append new data
    df_scaled = pd.concat([df_scaled, new_data_scaled])  # Keep growing dataset

    # Re-create sequences
    X_train, y_train = create_sequences(df_scaled, seq_length)

    # Fine-tune model with new data
    model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=1)

    # Save updated model
    model_json = model.to_json()
    weights = model.get_weights()
    with open(model_path, "wb") as f:
        pickle.dump((model_json, weights), f)

    print("Model updated with new data.")

# Get actual values entered by hospital staff
actual_monitors = float(input("Enter actual Patient Monitors required: "))
actual_defibrillators = float(input("Enter actual Defibrillators required: "))
actual_pumps = float(input("Enter actual Infusion Pumps required: "))

# Compare and update
print(f"Predicted: {predicted_equipment[0]:.2f}, Actual: {actual_monitors:.2f}")
print(f"Predicted: {predicted_equipment[1]:.2f}, Actual: {actual_defibrillators:.2f}")
print(f"Predicted: {predicted_equipment[2]:.2f}, Actual: {actual_pumps:.2f}")
update_model_with_actual([actual_monitors, actual_defibrillators, actual_pumps])


c:\Users\HARSHDIP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 38ms/step - loss: 0.0847 - val_loss: 0.0599
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0667 - val_loss: 0.0550
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0700 - val_loss: 0.0419
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0684 - val_loss: 0.0580
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0657 - val_loss: 0.0441
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0633 - val_loss: 0.0644
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0710 - val_loss: 0.0647
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0678 - val_loss: 0.0634
Epoch 9/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0629 - val_loss: 0.0546
Epoch 10/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0676 - val_loss: 0.0544
Epoch 11/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0600 - val_loss: 0.0424
Epoch 12/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step

In [3]:
from sklearn.metrics import r2_score

# Make predictions on the test set
y_pred_scaled = model.predict(X_test)

# Reverse scale the predictions
y_pred_full = np.zeros((len(y_pred_scaled), len(features) + len(targets)))
y_pred_full[:, -len(targets):] = y_pred_scaled  # Place predictions in target positions
y_pred = scaler.inverse_transform(y_pred_full)[:, -len(targets):]  # Extract only target values

# Reverse scale actual test values
y_test_full = np.zeros((len(y_test), len(features) + len(targets)))
y_test_full[:, -len(targets):] = y_test  # Place actual values in target positions
y_actual = scaler.inverse_transform(y_test_full)[:, -len(targets):]  # Extract only target values

# Compute R² score for each staff type
r2_doctors = r2_score(y_actual[:, 0], y_pred[:, 0])
r2_nurses = r2_score(y_actual[:, 1], y_pred[:, 1])
r2_technicians = r2_score(y_actual[:, 2], y_pred[:, 2])

# Display R² scores
print(f"R² Score for Doctors Prediction: {r2_doctors:.4f}")
print(f"R² Score for Nurses Prediction: {r2_nurses:.4f}")
print(f"R² Score for Technicians Prediction: {r2_technicians:.4f}")


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step  
R² Score for Doctors Prediction: 0.2106
R² Score for Nurses Prediction: -0.8418
R² Score for Technicians Prediction: 0.3263
